##Preparo el entorno

In [ ]:
!pip install diffusers transformers accelerate torch scipy gradio opencv-python -q


##Importo las librerías que vamos a utilizar

In [ ]:
# Librerías para procesamiento digital clásico
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Librerías para generación con IA
import torch
from diffusers import StableDiffusionXLPipeline

# Gradio
import gradio as gr


##Cargo el modelo de generación de imágenes

In [ ]:
# Cargar el modelo SDXL desde Hugging Face
#pipe = StableDiffusionXLPipeline.from_pretrained(
#    "CompVis/stable-diffusion-v1-4"
#)
#pipe.to("cpu")  # Usar la GPU
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
)
pipe.to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

##Función de generación de imagen con IA

In [ ]:
def generar_imagen(prompt, negative_prompt, steps, guidance, filtro):
    # Generar imagen con IA
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=steps,
        guidance_scale=guidance
    ).images[0]

    # Si no hay filtro, devolvés directamente la imagen generada
    if filtro == "Ninguno":
        return image

    # Convertir imagen PIL a formato OpenCV
    img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Aplicar filtro si corresponde
    if filtro == "Canny":
        img_cv = cv2.Canny(img_cv, 100, 200)
        return Image.fromarray(img_cv)  # Imagen en escala de grises

    elif filtro == "Blur":
        img_cv = cv2.GaussianBlur(img_cv, (5, 5), 0)
        img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
        return Image.fromarray(img_rgb)

    return image  # por si algo falla, devolvés la original


Esta función:

Toma un prompt positivo y uno negativo (para evitar cosas como “texto” o “blurry”)

Recibe dos parámetros: steps (cantidad de pasos del modelo) y guidance (qué tan fiel al texto debe ser)

Devuelve una imagen generada

##Creo interfaz

In [ ]:
gr.Interface(
    fn=generar_imagen,
    inputs=[
        gr.Textbox(label="Prompt positivo"),
        gr.Textbox(label="Prompt negativo (elementos a evitar)", placeholder="ej: text, blurry, distorted"),
        gr.Slider(10, 100, value=25, step=1, label="Número de pasos"),
        gr.Slider(1.0, 15.0, value=7.5, step=0.1, label="Guidance scale")
    ],
    outputs=gr.Image(type="pil", label="Imagen generada"),
    title="Generador de Imágenes con PDI + IA",
    description="Este generador te permite crear imágenes por texto y aplicar procesamiento digital con técnicas clásicas e inteligencia artificial."
).launch()


/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 5 arguments for function <function generar_imagen at 0x78cb01e631a0>, received 4.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 5 arguments for function <function generar_imagen at 0x78cb01e631a0>, received 4.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e22077295caa2880c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
